In [ ]:
!pip install ta

In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator, MACD
from ta.volatility import BollingerBands
from ta.volume import OnBalanceVolumeIndicator



In [18]:
# import GME data
df = yf.Ticker("GME")
df = GME.history(period="2y")
df.reset_index(inplace=True)
df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume
0,2023-09-11 00:00:00-04:00,17.850000,17.850000,16.900000,17.020000,3543800
1,2023-09-12 00:00:00-04:00,17.530001,18.500000,17.190001,17.230000,5179400
2,2023-09-13 00:00:00-04:00,17.309999,17.930000,17.270000,17.809999,3012600
3,2023-09-14 00:00:00-04:00,17.799999,18.520000,17.600000,18.330000,3078000
4,2023-09-15 00:00:00-04:00,18.360001,18.610001,18.150000,18.219999,6166600


In [25]:
# calculate the technical indicators
df['RSI'] = RSIIndicator(df['Close'], window=14).rsi()
df['SMA20'] = SMAIndicator(df['Close'], window=20).sma_indicator()
df['MACD'] = MACD(df['Close']).macd()
df['BB_high'] = BollingerBands(df['Close']).bollinger_hband()
df['BB_low'] = BollingerBands(df['Close']).bollinger_lband()
df['OBV'] = OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
df.dropna(inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume,RSI,SMA20,MACD,BB_high,BB_low,OBV
50,2023-11-20 00:00:00-05:00,13.00,13.20,12.80,12.80,2294900,44.597843,13.1910,-0.312538,14.198450,12.183550,-7740000
51,2023-11-21 00:00:00-05:00,12.65,12.72,12.26,12.55,3710100,42.378017,13.1165,-0.324005,14.081265,12.151735,-11450100
52,2023-11-22 00:00:00-05:00,12.56,12.71,12.19,12.29,2358800,40.140304,13.0455,-0.350038,14.033821,12.057179,-13808900
53,2023-11-24 00:00:00-05:00,12.22,12.41,12.18,12.20,1232500,39.365427,12.9805,-0.373624,14.010810,11.950190,-15041400
54,2023-11-27 00:00:00-05:00,12.18,12.33,11.89,11.91,3710600,36.894003,12.9200,-0.410980,14.047919,11.792081,-18752000


In [26]:
#create target column
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
df.head()

,Date,Open,High,Low,Close,Volume,RSI,SMA20,MACD,BB_high,BB_low,OBV,Target
50,2023-11-20 00:00:00-05:00,13.00,13.20,12.80,12.80,2294900,44.597843,13.1910,-0.312538,14.198450,12.183550,-7740000,0
51,2023-11-21 00:00:00-05:00,12.65,12.72,12.26,12.55,3710100,42.378017,13.1165,-0.324005,14.081265,12.151735,-11450100,0
52,2023-11-22 00:00:00-05:00,12.56,12.71,12.19,12.29,2358800,40.140304,13.0455,-0.350038,14.033821,12.057179,-13808900,0
53,2023-11-24 00:00:00-05:00,12.22,12.41,12.18,12.20,1232500,39.365427,12.9805,-0.373624,14.010810,11.950190,-15041400,0
54,2023-11-27 00:00:00-05:00,12.18,12.33,11.89,11.91,3710600,36.894003,12.9200,-0.410980,14.047919,11.792081,-18752000,1
